# Exercises Sheet 4 Search - Genetic Algorithms

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import time
random.seed(2)

class Game:
    def __init__(self, str_size):
        if str_size <= 0:
            raise ValueError("str_size must be a positive integer")
        self.str_size = str_size
        self.target_pattern = [random.choice((0,1)) for x in range(0,self.str_size)]

    def change_game_pattern(self):
        self.target_pattern = [random.choice((0,1)) for x in range(0,self.str_size)]

    def generate_random_bit_pattern(self):
        return [random.choice((0,1)) for x in range(0,self.str_size)]

    def is_correct_pattern(self, pattern):
        if len(pattern) != self.str_size:
            raise ValueError("Pattern size does not match the expected size")
        return pattern == self.target_pattern

    def mutate_pattern(self, pattern):
        if len(pattern) != self.str_size:
            raise ValueError("Pattern size does not match the expected size")

        position = random.randint(0, self.str_size - 1) # random position
        pattern_list[position] = 1 if pattern_list[position] == 0 else 1

        return mutated_pattern
        return pattern == self.target_pattern

    def calc_fitness(self, atual):
        if len(atual) != self.str_size:
            raise ValueError("Pattern size does not match the expected size")
        valor = 0
        for i in range(0,len(self.target_pattern)):
            if self.target_pattern[i] == atual[i]:
                valor += 1/len(self.target_pattern)
        
        return valor

def measure_performance(str_size, num_trials):

    attempts = []
    times = []
    fitness_lista = []

    for nada in range(num_trials):
        game = Game(str_size)
        start_time = time.time()
        attempt_count = 0

        while True:
            pattern = game.generate_random_bit_pattern()
            fitness = game.calc_fitness(pattern)
            fitness_lista.append(fitness)
            attempt_count += 1
            if game.is_correct_pattern(pattern):
                break

        print(fitness_lista)
        end_time = time.time()
        elapsed_time = end_time - start_time
        attempts.append(attempt_count)
        times.append(elapsed_time)

    return attempts, times

def plot_performance(str_sizes, num_trials):

    attempts_data = []
    times_data = []

    for str_size in str_sizes:
        attempts, times = measure_performance(str_size, num_trials)
        attempts_data.append(attempts)
        times_data.append(times)

    fig, axs = plt.subplots(2, 1, figsize=(10, 10))

    # Plot attempts
    axs[0].boxplot(attempts_data, positions=str_sizes, widths=0.6)
    axs[0].set_xlabel('Number of Bits')
    axs[0].set_ylabel('Number of Attempts')
    axs[0].set_title('Attempts vs Number of Bits')
    axs[0].set_xticks(str_sizes)

    # Plot times
    axs[1].boxplot(times_data, positions=str_sizes, widths=0.6)
    axs[1].set_xlabel('Number of Bits')
    axs[1].set_ylabel('Time (seconds)')
    axs[1].set_title('Time vs Number of Bits')
    axs[1].set_xticks(str_sizes)

    plt.tight_layout()
  
myGame = Game(6)
print(myGame.target_pattern)
print(myGame.generate_random_bit_pattern())
print(myGame.generate_random_bit_pattern())

#print(f"(num_attempts, time) for str_size=4: {measure_performance(str_size=4, num_trials=1)}")
#print(f"(num_attempts, time) for str_size=6: {measure_performance(str_size=6, num_trials=1)}")
#print(f"(num_attempts, time) for str_size=8: {measure_performance(str_size=8, num_trials=1)}")

# Define the pattern sizes to test#
#str_sizes = [2, 4, 8, 12, 16]#
# Plot the performance
#plot_performance(str_sizes, 30)

#measure_performance(12,100)
